In [1]:
import os
import fnmatch
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

C:\Users\Margo\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# Define the directory and variables of interest
path = "D:\\Thesis_Gl_Projections"
variables = ['lithk', 'acabf']
IS = 'GIS'
experiment_type = 'ctrl_proj'  # Experiment type you want to match

# Initialize a list to hold the matched file paths
matched_files = []

# Walk through the directory and find matched files
for root, dirs, files in os.walk(path):
    for filename in files:
        # Check if the file matches the required variable patterns, IS, and experiment type
        if (any(fnmatch.fnmatch(filename, f"{var}_*_*_*.nc") for var in variables) and 
                fnmatch.fnmatch(filename, f"*_{IS}*_*_{experiment_type}*.nc")):
            matched_files.append(os.path.join(root, filename))


In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr

# Initialize a dictionary to hold extracted data for PCA
variables = ['acabf', 'lithk']
extracted_data = {var: [] for var in variables}

# Loop through the matched files and extract data for the year 2015
for file in matched_files:
    # Extract model and group from the filename
    filename_parts = os.path.basename(file).split('_')
    group_name = filename_parts[2]  # Assuming group is in the 3rd position of the filename
    model_name = filename_parts[3]  # Assuming model is in the 4th position of the filename

    with xr.open_dataset(file, engine='netcdf4', use_cftime=True) as ds:
        # Extract the data for each variable
        for var in variables:
            if var in ds.variables:
                # Extract data for the variable (keeping all time steps)
                var_data = ds[var]  # This is a DataArray with dimensions (time, y, x)

                # Extract the data for the year 2015 (assuming it's the first index)
                year_index = 0  # Adjust if necessary
                year_data = var_data[year_index, :, :]  # Data for the year 2015

                # Change zeros to NaN
                year_data = year_data.where(year_data != 0, np.nan)  # Replace zeros with NaN

                # Convert the DataArray to a DataFrame and reset index
                year_df = year_data.to_dataframe().reset_index()

                # Drop rows where the variable is NaN
                year_df = year_df.dropna(subset=[var])

                # Add model and group information to the DataFrame
                year_df['model_group'] = model_name + "_" + group_name

                # Append the cleaned DataFrame for the year 2015 to the extracted data dictionary
                extracted_data[var].append(year_df)

# Optional: concatenate all DataFrames for each variable if needed
for var in extracted_data:
    if extracted_data[var]:  # Check if there is any data for the variable
        extracted_data[var] = pd.concat(extracted_data[var], ignore_index=True)  # Combine DataFrames
    else:
        extracted_data[var] = pd.DataFrame()  # Empty DataFrame if no data

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px

# Assuming `extracted_data` is a dictionary containing DataFrames for 'acabf' and 'lithk'
acabf_data = extracted_data['acabf']
lithk_data = extracted_data['lithk']

# Combine the 'acabf' and 'lithk' data into a single DataFrame
features = pd.DataFrame({
    'acabf': acabf_data['acabf'],
    'lithk': lithk_data['lithk'],
    'model_group': acabf_data['model_group']
})

# Check for NaN values and drop them
features.dropna(inplace=True)

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features[['acabf', 'lithk']])


# Perform PCA
pca = PCA(n_components=2)
components = pca.fit_transform(scaled_features)

# Prepare labels for the explained variance ratio
explained_variance = pca.explained_variance_ratio_ * 100
labels = {
    'PC1': f"PC1 ({explained_variance[0]:.1f}%)",
    'PC2': f"PC2 ({explained_variance[1]:.1f}%)"
}

# Create a DataFrame for the PCA components
components_df = pd.DataFrame(components, columns=labels.keys())

# Add model groups to the DataFrame
components_df['model_group'] = features['model_group'].values

# Optionally, add a label for each point
components_df['label'] = (
    features['acabf'].astype(str) + ', ' + features['lithk'].astype(str)
)


# Create a scatter plot using plotly.express
fig = px.scatter_matrix(
    components_df,
    dimensions=labels.keys(),
    color='model_group',
    symbol='model_group',
    title="PCA Scatter Matrix with Model Groups and Ice Thickness + ",
    labels=labels,
    hover_name='label',
    hover_data={
        'acabf': True,
        'lithk': True
    },
    opacity=0.6  # Add transparency to handle overlapping points
)

# Update the plot for better visualization
fig.update_traces(diagonal_visible=True)
fig.update_layout(showlegend=True)
fig.show()
